<a href="https://colab.research.google.com/github/lenchanti/TMD-framework/blob/main/unimodel_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import shutil
import re
import numpy as np
from scipy import interpolate
import pandas as pd
#import tensorflow_decision_forests as tfdf
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
import matplotlib.pyplot as plt
from sklearn import tree
from glob import glob
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

path="/content/drive/MyDrive/merge"


# z-변환 함수 정의
def z_normalize(data):
    return (data - np.mean(data)) / np.std(data)


ori_df=pd.DataFrame()
mode=pd.DataFrame()
for sample_files in os.listdir(path):

    sample_file=pd.read_csv(path+"//"+sample_files)

    #X=sample_file.iloc[:,:-2]
    # y=sample_file[['Mode', 'Survay1']]
    sample_mode_file=sample_file[['Mode']]
    # sample_mode_file=pd.get_dummies(sample_mode_file,columns=["Mode"])
    col_names=sample_file.iloc[:,1:-2].columns

    sample_file=sample_file.iloc[:,:-2]

    # 각 센서 데이터에 대해 z-변환 적용
    sample_file = sample_file.apply(z_normalize)
    sample_file=sample_file.iloc[:,7:]
    sample_file=sample_file.drop("Height",axis=1)
    sample_file=sample_file.drop("Proxi",axis=1)
    sample_file=sample_file.drop("Light",axis=1)
    sample_file=sample_file.drop("Step",axis=1)

    col_names=sample_file.columns
    ori_df=pd.concat([ori_df,sample_file],axis=0,ignore_index=True)
    mode=pd.concat([mode,sample_mode_file],
                      axis=0,ignore_index=True)



df=pd.DataFrame()
ori_df = ori_df.apply(z_normalize)

lst=[15,30 ,60,120,180,240,300]

for i in lst:
  df_mode=pd.DataFrame()
  df_mode = mode.iloc[::i].reset_index(drop=True)

  df=ori_df
  df_columns=df.columns



# 1. DataFrame을 NumPy 배열로 변환
  df = df.values


  df= np.reshape(df, (-1, i, 12))
# df= np.reshape(df, (-1, 15, 12))
# df= np.reshape(df, (-1, 30, 12))
# df= np.reshape(df, (-1, 60, 12))
#df= np.reshape(df, (-1, 120, 12))
# df= np.reshape(df, (-1, 180, 12))
# df= np.reshape(df, (-1, 240, 12))

# 결과 확인
  X=df
  y=df_mode
  y=pd.get_dummies(y,columns=["Mode"])





  epochs=30
  learning_rates=0.001
  batch_size=128


    #데이터 분할
  X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=101,stratify=y)




# 라벨을 one-hot 인코딩
#train_labels = to_categorical(y_train)
#test_labels = to_categorical(y_test)
#

# 2. CNN 모델 구축
  model = Sequential()
  model.add(Conv1D(filters=32, kernel_size=3, strides=1, padding='same', activation='relu', input_shape=(i,12)))

# 1st Max Pooling Layer (pool size 2, stride 2)
  model.add(MaxPooling1D(pool_size=2, strides=2))

# 2nd Convolutional Layer (64 filters, filter size 3, stride 1, padding same, ReLU activation)
  model.add(Conv1D(filters=64, kernel_size=3, strides=1, padding='same', activation='relu'))

# 2nd Max Pooling Layer (pool size 2, stride 2)
  model.add(MaxPooling1D(pool_size=2, strides=2))

# 3rd Convolutional Layer (128 filters, filter size 3, stride 1, padding same, ReLU activation)
  model.add(Conv1D(filters=128, kernel_size=3, strides=1, padding='same', activation='relu'))

# 3rd Max Pooling Layer (pool size 2, stride 2)
  model.add(MaxPooling1D(pool_size=2, strides=2))

# Flatten the output before passing to Dense layers
  model.add(Flatten())
  model.add(layers.Dense(256, activation='relu'))  # 세 번째 히든 레이어
  model.add(layers.Dropout(0.2))
  model.add(layers.Dense(7, activation='softmax'))

# 3. 모델 컴파일
  model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

# 4. 모델 학습
  model.fit(X_train, y_train, epochs=30, batch_size=128)


# 5. 테스트 데이터로 성능 평가
  test_loss, test_acc = model.evaluate(X_test, y_test)
  print(f"{i/60}s의 Test accuracy: {test_acc}")






Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
10474/10474 ━━━━━━━━━━━━━━━━━━━━ 121s 11ms/step - accuracy: 0.6796 - loss: 0.8305
Epoch 2/30
10474/10474 ━━━━━━━━━━━━━━━━━━━━ 139s 11ms/step - accuracy: 0.8391 - loss: 0.4461
Epoch 3/30
10474/10474 ━━━━━━━━━━━━━━━━━━━━ 150s 12ms/step - accuracy: 0.8709 - loss: 0.3656
Epoch 4/30
10474/10474 ━━━━━━━━━━━━━━━━━━━━ 135s 11ms/step - accuracy: 0.8853 - loss: 0.3275
Epoch 5/30
10474/10474 ━━━━━━━━━━━━━━━━━━━━ 144s 11ms/step - accuracy: 0.8954 - loss: 0.3009
Epoch 6/30
10474/10474 ━━━━━━━━━━━━━━━━━━━━ 115s 11ms/step - accuracy: 0.9014 - loss: 0.2835
Epoch 7/30
10474/10474 ━━━━━━━━━━━━━━━━━━━━ 144s 11ms/step - accuracy: 0.9064 - loss: 0.2710
Epoch 8/30
10474/10474 ━━━━━━━━━━━━━━━━━━━━ 122s 12ms/step - accuracy: 0.9110 - loss: 0.2583
Epoch 9/30
10474/10474 ━━━━━━━━━━━━━━━━━━━━ 138s 11ms/step - accuracy: 0.9138 - loss: 0.2501
Epoch 10/30
10474/10474 ━━━━━━━━━━━━━━━━━━━━ 142s 11ms/step - accuracy: 0.9171 - loss: 0.2419
Epoch 11/30
10474/10474 ━━━━━━━━━━━━━━━━━━━━ 142s 11ms/step - accurac

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
5237/5237 ━━━━━━━━━━━━━━━━━━━━ 106s 20ms/step - accuracy: 0.6665 - loss: 0.8584
Epoch 2/30
5237/5237 ━━━━━━━━━━━━━━━━━━━━ 100s 19ms/step - accuracy: 0.8229 - loss: 0.4827
Epoch 3/30
5237/5237 ━━━━━━━━━━━━━━━━━━━━ 144s 19ms/step - accuracy: 0.8575 - loss: 0.3950
Epoch 4/30
5237/5237 ━━━━━━━━━━━━━━━━━━━━ 139s 19ms/step - accuracy: 0.8753 - loss: 0.3493
Epoch 5/30
5237/5237 ━━━━━━━━━━━━━━━━━━━━ 147s 20ms/step - accuracy: 0.8855 - loss: 0.3226
Epoch 6/30
5237/5237 ━━━━━━━━━━━━━━━━━━━━ 139s 19ms/step - accuracy: 0.8924 - loss: 0.3037
Epoch 7/30
5237/5237 ━━━━━━━━━━━━━━━━━━━━ 144s 20ms/step - accuracy: 0.8989 - loss: 0.2862
Epoch 8/30
5237/5237 ━━━━━━━━━━━━━━━━━━━━ 140s 19ms/step - accuracy: 0.9031 - loss: 0.2735
Epoch 9/30
5237/5237 ━━━━━━━━━━━━━━━━━━━━ 103s 20ms/step - accuracy: 0.9077 - loss: 0.2597
Epoch 10/30
5237/5237 ━━━━━━━━━━━━━━━━━━━━ 103s 20ms/step - accuracy: 0.9103 - loss: 0.2541
Epoch 11/30
5237/5237 ━━━━━━━━━━━━━━━━━━━━ 147s 21ms/step - accuracy: 0.9127 - loss: 0.24

...

Epoch 1/30
873/873 ━━━━━━━━━━━━━━━━━━━━ 80s 89ms/step - accuracy: 0.5823 - loss: 1.0405
Epoch 2/30
873/873 ━━━━━━━━━━━━━━━━━━━━ 79s 86ms/step - accuracy: 0.7787 - loss: 0.5841
Epoch 3/30
873/873 ━━━━━━━━━━━━━━━━━━━━ 76s 87ms/step - accuracy: 0.8347 - loss: 0.4429
Epoch 4/30
873/873 ━━━━━━━━━━━━━━━━━━━━ 74s 85ms/step - accuracy: 0.8650 - loss: 0.3648
Epoch 5/30
873/873 ━━━━━━━━━━━━━━━━━━━━ 84s 86ms/step - accuracy: 0.8872 - loss: 0.3083
Epoch 6/30
873/873 ━━━━━━━━━━━━━━━━━━━━ 83s 87ms/step - accuracy: 0.9073 - loss: 0.2567
Epoch 7/30
873/873 ━━━━━━━━━━━━━━━━━━━━ 76s 87ms/step - accuracy: 0.9199 - loss: 0.2205
Epoch 8/30
873/873 ━━━━━━━━━━━━━━━━━━━━ 82s 87ms/step - accuracy: 0.9298 - loss: 0.1960
Epoch 9/30
873/873 ━━━━━━━━━━━━━━━━━━━━ 80s 84ms/step - accuracy: 0.9408 - loss: 0.1664
Epoch 10/30
873/873 ━━━━━━━━━━━━━━━━━━━━ 83s 86ms/step - accuracy: 0.9480 - loss: 0.1498
Epoch 11/30
873/873 ━━━━━━━━━━━━━━━━━━━━ 82s 86ms/step - accuracy: 0.9524 - loss: 0.1364
Epoch 12/30
873/873 ━━━━━━━━━━

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
655/655 ━━━━━━━━━━━━━━━━━━━━ 77s 115ms/step - accuracy: 0.5692 - loss: 1.0675
Epoch 2/30
655/655 ━━━━━━━━━━━━━━━━━━━━ 81s 114ms/step - accuracy: 0.7656 - loss: 0.6136
Epoch 3/30
655/655 ━━━━━━━━━━━━━━━━━━━━ 82s 114ms/step - accuracy: 0.8290 - loss: 0.4601
Epoch 4/30
655/655 ━━━━━━━━━━━━━━━━━━━━ 79s 110ms/step - accuracy: 0.8653 - loss: 0.3641
Epoch 5/30
655/655 ━━━━━━━━━━━━━━━━━━━━ 85s 114ms/step - accuracy: 0.8918 - loss: 0.2958
Epoch 6/30
655/655 ━━━━━━━━━━━━━━━━━━━━ 82s 114ms/step - accuracy: 0.9144 - loss: 0.2376
Epoch 7/30
655/655 ━━━━━━━━━━━━━━━━━━━━ 80s 112ms/step - accuracy: 0.9275 - loss: 0.2042
Epoch 8/30
655/655 ━━━━━━━━━━━━━━━━━━━━ 85s 116ms/step - accuracy: 0.9381 - loss: 0.1752
Epoch 9/30
655/655 ━━━━━━━━━━━━━━━━━━━━ 80s 114ms/step - accuracy: 0.9521 - loss: 0.1384
Epoch 10/30
655/655 ━━━━━━━━━━━━━━━━━━━━ 72s 110ms/step - accuracy: 0.9562 - loss: 0.1303
Epoch 11/30
655/655 ━━━━━━━━━━━━━━━━━━━━ 74s 113ms/step - accuracy: 0.9592 - loss: 0.1195
Epoch 12/30
655/655

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
524/524 ━━━━━━━━━━━━━━━━━━━━ 75s 139ms/step - accuracy: 0.5456 - loss: 1.1410
Epoch 2/30
524/524 ━━━━━━━━━━━━━━━━━━━━ 70s 134ms/step - accuracy: 0.7558 - loss: 0.6350
Epoch 3/30
524/524 ━━━━━━━━━━━━━━━━━━━━ 86s 141ms/step - accuracy: 0.8164 - loss: 0.4904
Epoch 4/30
524/524 ━━━━━━━━━━━━━━━━━━━━ 71s 135ms/step - accuracy: 0.8557 - loss: 0.3894
Epoch 5/30
524/524 ━━━━━━━━━━━━━━━━━━━━ 84s 139ms/step - accuracy: 0.8883 - loss: 0.3108
Epoch 6/30
524/524 ━━━━━━━━━━━━━━━━━━━━ 71s 135ms/step - accuracy: 0.9057 - loss: 0.2591
Epoch 7/30
524/524 ━━━━━━━━━━━━━━━━━━━━ 82s 135ms/step - accuracy: 0.9221 - loss: 0.2163
Epoch 8/30
524/524 ━━━━━━━━━━━━━━━━━━━━ 81s 132ms/step - accuracy: 0.9375 - loss: 0.1762
Epoch 9/30
524/524 ━━━━━━━━━━━━━━━━━━━━ 83s 134ms/step - accuracy: 0.9448 - loss: 0.1622
Epoch 10/30
524/524 ━━━━━━━━━━━━━━━━━━━━ 79s 129ms/step - accuracy: 0.9538 - loss: 0.1327
Epoch 11/30
524/524 ━━━━━━━━━━━━━━━━━━━━ 71s 136ms/step - accuracy: 0.9613 - loss: 0.1151
Epoch 12/30
524/524